In [213]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0...


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35113)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35113)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [233]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [217]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())
meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

## PCS5

In [205]:
pcs5 = load_schema.lobby(spark,
                     "pc",         
                     "live",      
                     "PickStatusChanged",  
                     "2021-09-08", 
                     "2021-09-30"  
                     )


In [206]:
pcs5_user = load_data_mart("pc", "2021-09-30", "2021-09-30", "user_master").where(col("lastlogindate") >= "2021-09-08").withColumnRenamed("accountid", "account_id")
pcs5_user = pcs5_user.withColumnRenamed("accountid", "account_id")

In [207]:
pcs5_user = pcs5_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pcs5_user = pcs5_user.join(meta_region, pcs5_user.country_new == meta_region.country_code_iso2, "left")

In [208]:
pcs5_coupon = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2021-09-08", "2021-09-30"). \
    where(col("Currency").like("%votingcoupon%"))

In [209]:
pcs5_coupon = pcs5_coupon.withColumnRenamed("AccountId", "account_id")
pcs5_coupon = pcs5_coupon.join(pcs5_user, "account_id", "left")

In [79]:
pcs5_coupon.where(col("Amount") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("owner count")).show(truncate=False)

+-----------+-----------+
|pubg_region|owner count|
+-----------+-----------+
|CN         |167613     |
|NA         |2675       |
|SA         |3264       |
|KR         |36595      |
|SEA        |14693      |
|JP         |688        |
|TW/HK      |1901       |
|CIS        |6402       |
|EMEA       |10757      |
|OC         |911        |
+-----------+-----------+



In [140]:
pcs5_user = pcs5_user.select("account_id", "pubg_region")
pcs5 = pcs5.join(pcs5_user, pcs5.AccountId == pcs5_user.account_id)

In [141]:
pcs5.groupBy("date").agg(count("*").alias("vote_cnt"), countDistinct("AccountId").alias("voter_cnt")).orderBy("date").toPandas()

,date,vote_cnt,voter_cnt
0,2021-09-08,1052606,86781
1,2021-09-09,630097,56092
2,2021-09-10,455475,42053
3,2021-09-11,421410,39537
4,2021-09-12,337938,32177
5,2021-09-13,244949,24293
6,2021-09-14,214136,21100
7,2021-09-15,202463,20291
8,2021-09-16,237282,25036
9,2021-09-17,288036,31572


In [77]:
pcs5.select(countDistinct("account_id").alias("total vote user cnt")).show()

+-------------------+
|total vote user cnt|
+-------------------+
|             586730|
+-------------------+



In [47]:
pcs5_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+-------+
|pubg_region|au     |
+-----------+-------+
|CN         |3576753|
|NA         |134887 |
|null       |1      |
|SA         |93907  |
|KR         |1116576|
|SEA        |220259 |
|Undefined  |97     |
|JP         |23967  |
|TW/HK      |46065  |
|CIS        |217876 |
|EMEA       |374403 |
|OC         |31035  |
+-----------+-------+



In [58]:
pcs5.groupBy("pubg_region").agg(countDistinct("AccountId").alias("vote_user_cnt"), count("*").alias("vote_cnt")).show(truncate=False)

+-----------+-------------+--------+
|pubg_region|vote_user_cnt|vote_cnt|
+-----------+-------------+--------+
|CN         |353881       |4783664 |
|NA         |6257         |84956   |
|SA         |8836         |127727  |
|KR         |116033       |1609694 |
|SEA        |42642        |618108  |
|Undefined  |1            |16      |
|JP         |1733         |24635   |
|TW/HK      |5079         |72773   |
|CIS        |20301        |283942  |
|EMEA       |29815        |424158  |
|OC         |2152         |29894   |
+-----------+-------------+--------+



In [94]:
pcs5.withColumn("vote_type", when(col("GameId").like("%BONUS%"), "BONUS").when(col("GameId").like("%BR%"), "BR")).groupBy("vote_type", "pubg_region").agg(count("*").alias("vote_cnt"), countDistinct("AccountId").alias("vote_user_cnt")).orderBy("vote_type").show(30, truncate=False)

+---------+-----------+--------+-------------+
|vote_type|pubg_region|vote_cnt|vote_user_cnt|
+---------+-----------+--------+-------------+
|BONUS    |EMEA       |403842  |28272        |
|BONUS    |Undefined  |16      |1            |
|BONUS    |TW/HK      |68847   |4736         |
|BONUS    |CN         |4465055 |317967       |
|BONUS    |JP         |23152   |1632         |
|BONUS    |CIS        |271702  |19500        |
|BONUS    |OC         |28141   |2019         |
|BONUS    |SEA        |589359  |40414        |
|BONUS    |KR         |1533054 |109999       |
|BONUS    |NA         |79999   |5852         |
|BONUS    |SA         |121640  |8370         |
|BR       |CIS        |12240   |5479         |
|BR       |SEA        |28749   |13294        |
|BR       |NA         |4957    |1808         |
|BR       |SA         |6087    |2672         |
|BR       |CN         |318609  |137092       |
|BR       |EMEA       |20316   |8548         |
|BR       |KR         |76640   |29521        |
|BR       |OC

In [97]:
gameid_region = pcs5.withColumn("vote_region", when(col("GameId").like("%ASIA%"), "ASIA").when(col("GameId").like("%APAC%"), "APAC").when(col("GameId").like("%AM%"), "AM").when(col("GameId").like("%EU%"), "EU")). \
    groupBy("vote_region", "pubg_region").agg(countDistinct("AccountId").alias("voter_cnt"), count("*").alias("vote_cnt")).toPandas()

In [99]:
gameid_region.sort_values("vote_region")

,vote_region,pubg_region,voter_cnt,vote_cnt
0,AM,Undefined,1,4
1,AM,OC,1984,7633
29,AM,CN,302596,1187878
41,AM,SA,8477,33636
26,AM,NA,5808,22374
24,AM,TW/HK,4651,18340
22,AM,SEA,39460,156312
42,AM,CIS,17979,71001
11,AM,KR,100557,394285
13,AM,JP,1553,6139


## PCS6

In [179]:
pcs6 = load_schema.lobby(spark,
                     "pc",         
                     "live",      
                     "PickStatusChanged",  
                     "2022-04-06", 
                     "2022-05-06"  
                     )

In [218]:
pcs6_coupon = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-03-16", "2022-05-06").where(col("Currency").like("%votingcoupon%"))

In [219]:
pcs6_coupon.select("Reason").distinct().show(truncate=False)

+----------------------+
|Reason                |
+----------------------+
|buy-cash-item         |
|by-support            |
|pickem-pick           |
|commercial-word-redeem|
|inventory-cleanup     |
|open-crate            |
|promotion-code-redeem |
+----------------------+



In [220]:
pcs6_user = load_data_mart("pc", "2022-05-06", "2022-05-06", "user_master").where(col("lastlogindate") >= "2022-04-06").withColumnRenamed("accountid", "account_id")
pcs6_membership = load_membership("pubg", "pc", "2022-05-06", "2022-05-06").select("account_id", "membership").distinct()
pcs6_user = pcs6_user.withColumnRenamed("accountid", "account_id")
pcs6_user = pcs6_user.join(pcs6_membership, "account_id")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38039)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38039)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [221]:
pcs6_user = pcs6_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pcs6_user = pcs6_user.join(meta_region, pcs6_user.country_new == meta_region.country_code_iso2, "left")

In [222]:
pcs6_coupon = pcs6_coupon.withColumnRenamed("AccountId", "account_id")
pcs6_coupon = pcs6_coupon.join(pcs6_user, "account_id", "left")
# pcs6_coupon.where(col("Amount") > 0).select(countDistinct("account_id").alias("total voting coupon owner count")).show()

In [196]:
pcs6_coupon_region = pcs6_coupon.where(col("Amount") > 0).groupBy("pubg_region", "membership").agg(countDistinct("account_id").alias("coupon_owner_cnt"), sum("Amount").alias("coupon_cnt")).toPandas()
pcs6_coupon_region.pivot_table(values="coupon_owner_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,1106,11873,701
CN,4031,90429,8022
EMEA,865,16301,1041
JP,16,461,23
KR,2120,20324,2449
NA,284,3672,256
OC,42,763,53
SA,328,4938,150
SEA,348,10333,276


In [227]:
pcs6_coupon_region_pick = pcs6_coupon.where(col("Reason")=="pickem-pick").groupBy("pubg_region", "membership").agg(countDistinct("account_id").alias("coupon_user_cnt"), sum("Amount").alias("coupon_use_cnt")).toPandas()

In [228]:
pcs6_coupon_region_pick.pivot_table(values="coupon_user_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,626,7317,522
CN,2036,63726,5823
EMEA,409,11080,643
JP,9,329,18
KR,1157,14424,1785
NA,102,2234,134
OC,16,518,35
SA,188,3457,108
SEA,211,8575,221


In [203]:
pcs6_coupon_region.pivot_table(values="coupon_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,1530,21019,1452
CN,6005,238175,22837
EMEA,1362,36862,2292
JP,23,1483,87
KR,4562,61697,7425
NA,495,9532,655
OC,84,2126,180
SA,448,10260,335
SEA,472,22136,644


In [232]:
pcs6_coupon_region_pick.pivot_table(values="coupon_use_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,-933,-13780,-1032
CN,-2883,-127561,-12070
EMEA,-616,-22959,-1201
JP,-14,-844,-48
KR,-1826,-31389,-3676
NA,-162,-5158,-300
OC,-27,-1223,-73
SA,-259,-6662,-205
SEA,-285,-16361,-448


In [197]:
pcs6_user = pcs6_user.select("account_id", "membership", "pubg_region")
pcs6 = pcs6.join(pcs6_user, pcs6.AccountId == pcs6_user.account_id)

In [134]:
pcs6_daily_vote = pcs6.groupBy("date", "membership").agg(count("*").alias("vote_cnt"), countDistinct("AccountId").alias("voter_cnt")).toPandas()

In [136]:
pcs6_daily_vote.pivot_table(values = "voter_cnt", columns="membership", index="date")

membership,basic,legacy,plus
date,,,
2022-04-06,13612,111500,9441
2022-04-07,6602,48279,3407
2022-04-08,5806,35918,2350
2022-04-09,5376,31537,1887
2022-04-10,4354,24404,1413
2022-04-11,3594,17347,1032
2022-04-12,3795,16297,932
2022-04-13,1915,12303,676
2022-04-14,1716,10104,614


In [93]:
pcs6_user.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                  10323773|
+--------------------------+



In [85]:
pcs6.select(countDistinct("account_id").alias("total voter count")).show()

+-----------------+
|total voter count|
+-----------------+
|           632904|
+-----------------+



In [23]:
pcs6_user.groupBy("membership").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+----------+-------+
|membership|au     |
+----------+-------+
|plus      |174226 |
|basic     |4723542|
|legacy    |5426005|
+----------+-------+



In [68]:
pcs6.groupBy("membership").agg(countDistinct("AccountId").alias("vote_user_cnt"), count("*").alias("vote_cnt")).show(truncate=False)

+----------+-------------+--------+
|membership|vote_user_cnt|vote_cnt|
+----------+-------------+--------+
|plus      |32109        |315387  |
|basic     |84996        |849373  |
|legacy    |515799       |5236216 |
+----------+-------------+--------+



In [19]:
pcs6.withColumn("vote_type", when(col("GameId").like("%BONUS%"), "BONUS").when(col("GameId").like("%BR%"), "BR")).groupBy("vote_type", "membership").agg(count("*").alias("vote_cnt"), countDistinct("AccountId").alias("vote_user_cnt")).show(truncate=False)

+---------+----------+--------+-------------+
|vote_type|membership|vote_cnt|vote_user_cnt|
+---------+----------+--------+-------------+
|BONUS    |plus      |296176  |28462        |
|BR       |plus      |19211   |9358         |
|BR       |basic     |7063    |4792         |
|BONUS    |legacy    |5007808 |480492       |
|BR       |legacy    |228408  |112726       |
|BONUS    |basic     |842310  |82909        |
+---------+----------+--------+-------------+



In [29]:
pcs6_user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au")).show(truncate=False)

+-----------+-------+
|pubg_region|au     |
+-----------+-------+
|CN         |5756481|
|NA         |329162 |
|null       |1      |
|SA         |297768 |
|KR         |1365636|
|SEA        |514739 |
|Undefined  |1389   |
|JP         |43369  |
|TW/HK      |89145  |
|CIS        |874111 |
|EMEA       |1065741|
|OC         |47606  |
+-----------+-------+



In [69]:
pcs6.groupBy("pubg_region").agg(countDistinct("AccountId").alias("vote_user_cnt"), count("*").alias("vote_cnt")).show(truncate=False)

+-----------+-------------+--------+
|pubg_region|vote_user_cnt|vote_cnt|
+-----------+-------------+--------+
|CN         |373696       |3786659 |
|NA         |8575         |82543   |
|SA         |13664        |142613  |
|KR         |94448        |935895  |
|SEA        |38360        |416027  |
|Undefined  |1            |7       |
|JP         |1625         |17026   |
|TW/HK      |5738         |59248   |
|CIS        |43614        |434048  |
|EMEA       |51066        |505646  |
|OC         |2117         |21264   |
+-----------+-------------+--------+



In [167]:
region_membership_voter = pcs6.groupBy("pubg_region", "membership").agg(countDistinct("AccountId").alias("voter_cnt")).toPandas()
region_membership_voter.pivot_table(values="voter_cnt", columns="membership", index="pubg_region")

membership,basic,legacy,plus
pubg_region,,,
CIS,10255.0,31797.0,1562.0
CN,47566.0,305278.0,20852.0
EMEA,7307.0,41708.0,2051.0
JP,149.0,1432.0,44.0
KR,12571.0,75879.0,5998.0
NA,1369.0,6850.0,356.0
OC,194.0,1847.0,76.0
SA,1857.0,11505.0,302.0
SEA,3125.0,34551.0,684.0


In [198]:
pcs6_pickem_region = pcs6.withColumn("vote_type", when(col("GameId").like("%BONUS%"), "BONUS").when(col("GameId").like("%BR%"), "BR")). \
    groupBy("vote_type", "pubg_region", "membership").agg(count("*").alias("vote_cnt"), countDistinct("AccountId").alias("vote_user_cnt")).orderBy("vote_type").toPandas()

In [200]:
pcs6_pickem_region[pcs6_pickem_region.vote_type=="BR"].pivot_table(values="vote_user_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,626,7317,522
CN,2036,63726,5823
EMEA,409,11080,643
JP,9,329,18
KR,1157,14424,1785
NA,102,2234,134
OC,16,518,35
SA,188,3457,108
SEA,211,8575,221


In [204]:
pcs6_pickem_region[pcs6_pickem_region.vote_type=="BR"].pivot_table(values="vote_cnt", index="pubg_region", columns="membership")

membership,basic,legacy,plus
pubg_region,,,
CIS,933,13780,1032
CN,2883,127561,12070
EMEA,616,22959,1201
JP,14,844,48
KR,1826,31389,3676
NA,162,5158,300
OC,27,1223,73
SA,259,6662,205
SEA,285,16360,448


In [100]:
gameid_region_pcs6 = pcs6.withColumn("vote_region", when(col("GameId").like("%ASIA%"), "ASIA").when(col("GameId").like("%APAC%"), "APAC").when(col("GameId").like("%AM%"), "AM").when(col("GameId").like("%EU%"), "EU")). \
    groupBy("vote_region", "pubg_region").agg(countDistinct("AccountId").alias("voter_cnt"), count("*").alias("vote_cnt")).toPandas()

In [101]:
gameid_region_pcs6

,vote_region,pubg_region,voter_cnt,vote_cnt
0,AM,OC,1544,5936
1,APAC,CIS,32273,122153
2,EU,SEA,24531,98317
3,ASIA,TW/HK,3628,14687
4,EU,EMEA,29352,117974
5,ASIA,NA,4530,17836
6,EU,CN,231702,925266
7,APAC,KR,69040,261082
8,EU,OC,1158,4643
9,AM,KR,67198,256233


In [106]:
gameid_region_pcs6_membership = pcs6.withColumn("vote_region", when(col("GameId").like("%ASIA%"), "ASIA").when(col("GameId").like("%APAC%"), "APAC").when(col("GameId").like("%AM%"), "AM").when(col("GameId").like("%EU%"), "EU")). \
    groupBy("vote_region", "pubg_region", "membership").agg(countDistinct("AccountId").alias("voter_cnt"), count("*").alias("vote_cnt")).toPandas()

In [111]:
gameid_region_pcs6_membership.sort_values(["vote_region", "pubg_region","membership"])[31:60]

,vote_region,pubg_region,membership,voter_cnt,vote_cnt
2,APAC,CIS,legacy,23660,89974
67,APAC,CIS,plus,1201,4588
7,APAC,CN,basic,37255,141155
46,APAC,CN,legacy,199886,754574
80,APAC,CN,plus,14914,55168
20,APAC,EMEA,basic,4981,18393
62,APAC,EMEA,legacy,29335,112025
91,APAC,EMEA,plus,1541,5633
95,APAC,JP,basic,89,334
13,APAC,JP,legacy,964,3701


In [112]:
gameid_region_pcs6_membership.pivot_table(values="voter_cnt", index=["pubg_region", "membership"], columns=["vote_region"])

vote_region                   AM      APAC      ASIA        EU
pubg_region membership                                        
CIS         basic         7170.0    7412.0    4436.0    4519.0
            legacy       23097.0   23660.0   18468.0   18760.0
            plus          1153.0    1201.0     882.0     912.0
CN          basic        35984.0   37255.0   31966.0   30629.0
            legacy      196202.0  199886.0  203408.0  189297.0
            plus         13998.0   14914.0   12873.0   11776.0
EMEA        basic         4922.0    4981.0    3014.0    3142.0
            legacy       29704.0   29335.0   24488.0   25173.0
            plus          1473.0    1541.0    1009.0    1037.0
JP          basic           88.0      89.0      96.0      85.0
            legacy         967.0     964.0     976.0     929.0
            plus            33.0      36.0      25.0      23.0
KR          basic         8531.0    8930.0    6132.0    5745.0
            legacy       54381.0   55565.0   44826.0   42654.0
            plus          4286.0    4545.0    3302.0    3128.0
NA          basic          990.0     983.0     724.0     719.0
            legacy        5143.0    4740.0    3662.0    3547.0
            plus           259.0     243.0     144.0     145.0
OC          basic          124.0     125.0      94.0      89.0
            legacy        1366.0    1359.0    1075.0    1038.0
            plus            54.0      61.0      31.0      31.0
SA          basic         1310.0    1285.0     807.0     793.0
            legacy        9063.0    8422.0    7005.0    6977.0
            plus           223.0     215.0     172.0     168.0
SEA         basic         2006.0    2187.0    1525.0    1493.0
            legacy       24610.0   26543.0   23275.0   22628.0
            plus           464.0     521.0     423.0     410.0
TW/HK       basic          397.0     402.0     301.0     283.0
            legacy        3467.0    3447.0    3227.0    3065.0
            plus           132.0     141.0     100.0      94.0
Undefined   basic            NaN       NaN       1.0       1.0

In [107]:
# pcs5_tmp = pcs5.withColumn("vote_region", when(col("GameId").like("%ASIA%"), "ASIA").when(col("GameId").like("%APAC%"), "APAC").when(col("GameId").like("%AM%"), "AM").when(col("GameId").like("%EU%"), "EU")). \
#     select("AccountId", "vote_region", "pubg_region").withColumn("esport", lit("pcs5"))
# pcs6_tmp = pcs6.withColumn("vote_region", when(col("GameId").like("%ASIA%"), "ASIA").when(col("GameId").like("%APAC%"), "APAC").when(col("GameId").like("%AM%"), "AM").when(col("GameId").like("%EU%"), "EU")). \
#     select("AccountId", "vote_region", "pubg_region", "membership").withColumn("esport", lit("pcs5"))